<a href="https://colab.research.google.com/github/benjaminsinzore/ASP.NET-MYSQL/blob/main/Fine_Tuning_Moshi_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with Fine-Tuning Moshi 7B

This notebook shows you a simple example of how to LoRA finetune Moshi 7B. You can run this notebook in Google Colab using a A100 GPU.

<a target="_blank" href="https://colab.research.google.com/github//kyutai-labs/moshi-finetune/blob/main/tutorials/moshi_finetune.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Check out `moshi-finetune` Github repo to learn more: https://github.com/kyutai-labs/moshi-finetune/


## Installation

Clone the `moshi-finetune` repo:


In [1]:
%cd /content/
!git clone https://github.com/kyutai-labs/moshi-finetune.git

/content
Cloning into 'moshi-finetune'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 227 (delta 28), reused 24 (delta 24), pack-reused 190 (from 1)
Receiving objects: 100% (227/227), 623.92 KiB | 7.01 MiB/s, done.
Resolving deltas: 100% (127/127), done.


Install all required dependencies:


In [2]:
%pip install -e /content/moshi-finetune

Obtaining file:///content/moshi-finetune
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/kyutai-labs/moshi.git to /tmp/pip-install-1nc0qg6s/moshi_57d0eb88241e4a2382a5fbceb0a5c9e7
  Running command git clone --filter=blob:none --quiet https://github.com/kyutai-labs/moshi.git /tmp/pip-install-1nc0qg6s/moshi_57d0eb88241e4a2382a5fbceb0a5c9e7
  Resolved https://github.com/kyutai-labs/moshi.git to commit 62d0154eb199074c459f1fef2ef71028486fd528
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Prepare dataset


In [ ]:
from pathlib import Path
from huggingface_hub import snapshot_download
import time

Path("/content/data/daily-talk-contiguous").mkdir(parents=True, exist_ok=True)

# Download the dataset with retries and delay
local_dir = None
retries = 3  # Number of retries
delay = 5    # Delay in seconds between retries

for i in range(retries):
    try:
        local_dir = snapshot_download(
            "kyutai/DailyTalkContiguous",
            repo_type="dataset",
            local_dir="/content/data/daily-talk-contiguous",
        )
        break  # Exit loop if successful
    except Exception as e:
        if "429" in str(e):  # Check for rate limit error
            print(f"Rate limit hit. Retrying in {delay} seconds... (Attempt {i+1}/{retries})")
            time.sleep(delay)
        else:
            raise e  # Re-raise other exceptions

if local_dir is None:
    print("Failed to download dataset after multiple retries.")
else:
    print("Dataset downloaded successfully!")

Fetching 5085 files:   0%|          | 0/5085 [00:00<?, ?it/s]

1198.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

1199.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

1197.json:   0%|          | 0.00/4.11k [00:00<?, ?B/s]

1198.wav:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

...

1284.wav:   0%|          | 0.00/5.25M [00:00<?, ?B/s]

1285.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

1285.wav:   0%|          | 0.00/6.14M [00:00<?, ?B/s]

1286.wav:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

1287.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

## Start training


In [ ]:
# these info is needed for training
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters
import yaml

config = """
# data
data:
  train_data: '/content/data/daily-talk-contiguous/dailytalk.jsonl' # Fill
  eval_data: '' # Optionally Fill
  shuffle: true

# model
moshi_paths:
  hf_repo_id: "kyutai/moshiko-pytorch-bf16"


full_finetuning: false # Activate lora.enable if partial finetuning
lora:
  enable: true
  rank: 128
  scaling: 2.
  ft_embed: false

# training hyperparameters
first_codebook_weight_multiplier: 100.
text_padding_weight: .5


# tokens per training steps = batch_size x num_GPUs x duration_sec
# we recommend a sequence duration of 300 seconds
# If you run into memory error, you can try reduce the sequence length
duration_sec: 100
batch_size: 1
max_steps: 300

gradient_checkpointing: true # Activate checkpointing of layers

# optim
optim:
  lr: 2.e-6
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 10
eval_freq: 1
do_eval: False
ckpt_freq: 10

save_adapters: True

run_dir: "/content/test"  # Fill
"""

# save the same file locally into the example.yaml file
with open("/content/example.yaml", "w") as file:
    yaml.dump(yaml.safe_load(config), file)

In [ ]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test

In [ ]:
# start training

!cd /content/moshi-finetune && torchrun --nproc-per-node 1 -m train /content/example.yaml

## Inference

Once the model has been trained, inference can be run on the colab GPU too, and gradio can be used to tunnel the audio data from a local client to the notebook.

More details on how to set this up can be found in the [moshi readme](https://github.com/kyutai-labs/moshi?tab=readme-ov-file#python-pytorch).


In [ ]:
!pip install gradio

In [ ]:
!python -m moshi.server --gradio-tunnel --lora-weight=/content/test/checkpoints/checkpoint_000300/consolidated/lora.safetensors --config-path=/content/test/checkpoints/checkpoint_000300/consolidated/config.json